In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams 
from nltk.tokenize import sent_tokenize
from collections import Counter
import regex as re 
from numpy import dot
from numpy.linalg import norm
import numpy as np 
import json 
import random 
import spacy 
import tqdm 

In [2]:
nlp = spacy.load('xx_sent_ud_sm')
nlp.add_pipe('sentencizer')

In [ ]:
label_map = {} 
with open('/scratch/useful/subject_set_labels.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        label_map.update(line)

In [ ]:
prop_dict = {} 
with open('/scratch/useful/ontology_props.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        prop_dict.update({line['name']: line['properties']})

In [ ]:
type_dict = {} 
with open('/scratch/useful/mapping_transitive.ttl', 'r') as f:
    for line in f:
        line = line.split()
        if(line[0] not in type_dict):
            type_dict[line[0]] = set() 
        type_dict[line[0]].add(line[2].split('/')[-1])

In [ ]:
type_dict['<http://dbpedia.org/resource/!!!>']

{'Agent>',
 'DUL.owl#Agent>',
 'DUL.owl#SocialPerson>',
 'Group>',
 'MusicGroup>',
 'Organisation>',
 'Organization>',
 'Q215380>',
 'Q24229398>',
 'Q43229>',
 'owl#Thing>'}

In [ ]:
def get_prop_name(item):
    if("#literal" in item):
        return item.split('#literal')[0], True
    if(item in label_map):
        tail_name = label_map[item]
    else:
        tail_name = re.sub('_', ' ',item.split('/')[-1])
    return tail_name, False

In [ ]:
wanted_types = ["Place", "Person", "Organization", "Organisation"]

In [ ]:
langs = ['ga', 'de', 'en']

In [ ]:
candidates = {}

In [ ]:
for lang in langs:
    print(lang)
    with(open(f'/scratch/useful/abstracts_{lang}.jsonl', 'r')) as f:
        pb = tqdm.tqdm(total=1000000)
        for i, line in enumerate(f):
            pb.update(1)
            item = json.loads(line)
            rsc = item['resource']
            txt = item['text']
            found = False 

            for t in wanted_types:
                if(f'{t}>' in type_dict[f'<{rsc}>']):
                    found = True 
            
            if(not(found)):
                continue  

            name = get_prop_name(rsc)[0]
            if(name not in candidates):
                candidates[name] = {}

                properties = [] 
                fw_props = prop_dict[rsc]['properties']
                for prop in fw_props:
                    for item in fw_props[prop]:
                        item_name = get_prop_name(item)[0]
                        properties.append((name, prop, item_name))
                
                rv_props = prop_dict[rsc]['reverse_properties']
                for prop in rv_props:
                    for item in rv_props[prop][:3]:
                        item_name = get_prop_name(item)[0]
                        properties.append((item_name, prop, name))
                
                candidates[name]['properties'] = properties

            doc = nlp(txt)
            filtered_sents = []
            for sent in doc.sents:
                sent_ = sent.text
                if(len(sent_.split()) > 5 and len(sent_.split()) < 250):
                    filtered_sents.append(sent_)
            candidates[name][f'{lang}_text'] = filtered_sents

ga


  0%|          | 0/1000000 [00:00<?, ?it/s]

NameError: name 'wanted_types' is not defined

In [ ]:
save_name = '/scratch/useful/filtered_candidates.json'
with open(save_name, 'w') as f:
    json.dump(candidates, f)

In [11]:
candidates = json.load(open('/scratch/candidates.json', 'r'))

In [12]:
de_sents = open('../scripts/translated_sents_de.txt').readlines() 
ga_sents = open('../scripts/translated_sents_ga.txt').readlines()

de_pointer = 0 
ga_pointer = 0

In [13]:
for entity in candidates:
    if('de_text' in candidates[entity]):
        print(entity)
        candidates[entity]['translated_sents'] = de_sents[de_pointer:de_pointer+len(candidates[entity]['de_text'])]
        de_pointer += len(candidates[entity]['de_text'])
        break 
    # if('ga_text' in candidates[entity]):
    #     candidates[entity]['translated_sents'] = ga_sents[ga_pointer:ga_pointer+len(candidates[entity]['ga_text'])]
    #     ga_pointer += len(candidates[entity]['ga_text'])

\"Weird Al\" Yankovic


In [17]:
candidate_keys = list(candidates.keys())

In [22]:
candidates[candidate_keys[564]]['de_text']

NameError: name 'candidate_keys' is not defined

In [10]:
de_pointer, len(de_sents), ga_pointer, len(ga_sents)

(2800843, 1587650, 76978, 50678)